In [1]:
import os
import pandas as pd

from CGRtools import RDFRead

from Synto.chem.reaction_rules.extracted_rules.transformations import ExtractRule, ReverseReaction

from Synto.chem.reaction_rules.extracted_rules.filters import (CheckStrangeCarbons, 
                                                               CheckNoReaction,
                                                               CheckMultiCenterReaction,
                                                               CheckWrongCHBreaking,
                                                               CheckCCsp3Breaking,
                                                               CheckCCRingBreaking,
                                                               CheckRulesByPopularity)

from Synto.chem.reaction_rules.extracted_rules.filters import (CheckCGRConnectedComponents,
                                                               IsCompeteProducts,
                                                               CheckDynamicBondsNumber,
                                                               CheckSmallMolecules,
                                                               CheckRings)

from Synto.chem.reaction_rules.extracted_rules.processing import apply_transformations

from CGRtools.containers import MoleculeContainer
from CGRtools import CGRReactor, Reactor 

from IPython.display import SVG, display

In [2]:
filters = [CheckStrangeCarbons(),
           CheckNoReaction(),
           CheckMultiCenterReaction(),
           CheckWrongCHBreaking(),
           CheckCCsp3Breaking(),
           CheckCCRingBreaking(),
           CheckCGRConnectedComponents(),
           CheckDynamicBondsNumber(),
           CheckSmallMolecules()]

## Load data

In [3]:
data = RDFRead('uspto_subset.rdf').read()

## Rule extractor

In [4]:
rule_extractor = ExtractRule()

## Rule extraction

In [5]:
reaction = data[249]
print([filt(reaction) for filt in filters])
reaction

[False, False, False, False, False, False, False, False, False]


In [6]:
extracted_rules = rule_extractor(reaction)
print(f'Num of extracted reaction rules: {len(extracted_rules)}\n')

forward_rule = extracted_rules[0]
print(forward_rule)
forward_rule

Num of extracted reaction rules: 1

[C]-[C;D2;H2;Zs]-[C].[C]-[C;D3;H0;Zd](=[O])-[O;D2;H0;Zs]-[C]>[C;D1;H3;Zs]-[C;D2;H2;Zs]-[O;D1;H0;Zs-].[Na;D0;H0;Zs+].[Na;D0;H0;Zs]>[C]-[C;D3;H1;Zs](-[C])-[C;D3;H0;Zd](-[C])=[O]


Apply the extracted reaction rule to the reactants:

In [7]:
forward_reactor = CGRReactor(forward_rule) 

First we need to merge reactants

In [8]:
if len(reaction.reactants) == 2:
    merged_reactants = MoleculeContainer.union(reaction.reactants[0], reaction.reactants[1])
else:
    merged_reactants = reaction.reactants[0]
merged_reactants

In [9]:
reaction

In [10]:
generated_products = list(forward_reactor(merged_reactants))

try:
    generated_products = generated_products[0].split()
except:
    pass

print(f'Num of products: {len(generated_products)}\n')

for i in generated_products:
    i.clean2d()
    
generated_products[0]

Num of products: 1



## Forward and retro reaction rules

In [11]:
forward_rule

Convert forward reaction rule to the retro reaction rule

In [12]:
reverse_reaction = ReverseReaction()
retro_rule = reverse_reaction(forward_rule)
retro_rule

Apply retro reaction rule to the product to get the reactants

In [13]:
retro_reactor = CGRReactor(retro_rule)

In [14]:
if len(reaction.products) == 2:
    merged_products = MoleculeContainer.union(reaction.products[0], reaction.products[1])
else:
    merged_products = reaction.products[0]

generated_reactant_pairs = list(retro_reactor(merged_products))
print(f'Num of reactants pairs: {len(generated_reactant_pairs)}')

Num of reactants pairs: 2


In [15]:
generated_reactants = generated_reactant_pairs[0].split()
for i in generated_reactants:
    i.clean2d()

In [16]:
reverse_reaction(reaction)

In [17]:
generated_reactants[0]

In [18]:
generated_reactants[1]